In [57]:
import sys
from pyspark import SparkConf, SparkContext
from operator import itemgetter
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)
lines = sc.textFile("browsing.txt")

products = lines.flatMap(lambda x: x.split(" ")).countByValue()
pro_100={}
for key,value in products.items():#single element which appears more than 100
    if value>=100:
         pro_100.setdefault(str(key),int(value))
            
def split_line(line):#split every line in to separate elements
    pros=[]
    for pro in line.split():
        pros.append(str(pro))
    return pros
def pair_product(p):
    pairs=[]
    n=len(p)
    for i in range(0,n-1):
        for j in range(i+1,n):
            if (p[i] in pro_100) and (p[j] in pro_100):
                pairs.append(((p[i],p[j]),1))
                pairs.append(((p[j],p[i]),1))
    return pairs
def confidence(p):
    sup=int(pro_100[p[0][0]])
    conf=p[1]/sup
    return (p[0],conf)
pro=lines.map(split_line)
p_pairs=pro.flatMap(pair_product)
counted=p_pairs.groupByKey() \
    .map(lambda x: (x[0], sum(x[1])))\
    .filter(lambda x: x[1] >= 100)
result=counted.map(confidence)
result=result.collect()
result=sorted(result, key=itemgetter(1,0),reverse=True)
print(result[0:5])
sc.stop()

[(('DAI93865', 'FRO40251'), 1.0), (('GRO85051', 'FRO40251'), 0.999176276771005), (('GRO38636', 'FRO40251'), 0.9906542056074766), (('ELE12951', 'FRO40251'), 0.9905660377358491), (('DAI88079', 'FRO40251'), 0.9867256637168141)]
